In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
from glob import glob
from time import time
from itertools import product
pd.set_option("display.max_rows", 500)
%matplotlib inline
slicer = pd.IndexSlice

In [ ]:
OUTPUT_DIR = '/app/output'

In [ ]:
csv_files_list = glob(os.path.join(OUTPUT_DIR, 'FP-FN_grid_search_*.csv'))
csv_files_list = csv_files_list[:100]
len(csv_files_list)

In [ ]:
penalizers_results = []
run_numbers = []
for idf, file in enumerate(csv_files_list):
    gauc_cv_results = pd.read_csv(file, index_col=[0,1])
    penalizers_results.append(np.log(gauc_cv_results['Mean'].idxmax()))
    run_numbers.append(int(file.split('grid_search_')[1].replace('.csv', '')))

results_df = pd.DataFrame(penalizers_results, index=run_numbers).sort_index()
results_df.head()

In [ ]:
results_df.mean(axis=0)

In [ ]:
results_df.std(axis=0).round(3)

In [ ]:
j_events = 2
n_folds = 5

In [ ]:
results_df

In [ ]:
runs_tp_fp = pd.DataFrame(index=pd.MultiIndex.from_tuples(list(product(results_df.index, [1,2]))), columns=['TP_mean', 'FP_mean'])
for idr, row in results_df.iterrows():
    with open(os.path.join(OUTPUT_DIR, f'FP-FN_grid_search_{idr}.pkl'), 'rb') as f:
        grid_search = pickle.load(f)

        for risk in range(1, j_events+1):
            n_TP, n_FP = [], [] 
            for i_fold in range(n_folds):
                params = grid_search.folds_grids[i_fold].meta_models[np.exp(row[risk-1].round(5))].beta_models[risk].params_
                n_TP.append((params.iloc[:5].round(4).abs() > 0).sum())
                n_FP.append((params.iloc[5:].round(4).abs() > 0).sum())
                
            runs_tp_fp.loc[(idr, risk), 'TP_mean'] = np.mean(n_TP)
            runs_tp_fp.loc[(idr, risk), 'FP_mean'] = np.mean(n_FP)

In [ ]:
runs_tp_fp.loc[slicer[:, 1], ['TP_mean', 'FP_mean']].mean(axis=0)

In [ ]:
runs_tp_fp.loc[slicer[:, 2], ['TP_mean', 'FP_mean']].mean(axis=0)